In [3]:
# create list
file2 = "common_nodes_hPPIN_STRING_PANCAN.txt"
combined_list = []
with open(file2) as IN:
    for line in IN:
        combined_list.append(line.rstrip().split("\t")[0])

In [26]:
# using ast.literal_eval()
import sys
import ast
import os
import time
import pandas as pd
import pickle
import glob
import numpy as np
import statistics
from scipy.stats import mannwhitneyu

In [27]:
with open("/data/timonaj/cancer_as_wound/wgcna_tutorial/weighted_pan_cancer_paths.dictionary", 'rb') as config_dictionary_file:
 
    # Step 3
    paths = pickle.load(config_dictionary_file)

In [28]:
def compute_sp(list1, list2, paths):
    total_sp = []
    for i in range(len(list1)):
        for j in range(len(list2)):
            if(list1[i] in paths.keys()):
                shortest_paths = paths[list1[i]]
                if(list2[j] in shortest_paths.keys()):
                    total_sp.append(shortest_paths[list2[j]])
    return({"distribution": total_sp,
            "mean": statistics.mean(total_sp),
            "median": statistics.median(total_sp)})

In [29]:
# Read Cancersea Data
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/cancersea/"

filenames = os.listdir(folder_path)

cancersea = {}

for filename in filenames:
    file_path = f"{folder_path}/{filename}"
    cancersea[filename[:-4]] = list()

    with open(file_path) as IN:
        for line in IN:
            cancersea[filename[:-4]].append(line.rstrip().split("\t")[1])

In [30]:
cancersea["COSMIC"] = list()
with open("/data/timonaj/cancer_as_wound/ppi_analysis/cosmic_hallmarks.txt") as IN:
    for line in IN:
        cancersea["COSMIC"].append(line.rstrip().split("\t")[0])

In [31]:
# read in HP sets
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets/"

filenames = os.listdir(folder_path)

hp_genesets = {}

for filename in filenames:
    file_path = f"{folder_path}/{filename}"
    hp_genesets[filename[:-4]] = list()

    with open(file_path) as IN:
        for line in IN:
            hp_genesets[filename[:-4]].append(line.rstrip().split("\t")[0])

In [32]:
# read in HP matched controls set
folder_path = "/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets_matched_controls/"

filenames = glob.glob("/data/timonaj/cancer_as_wound/ppi_analysis/HP_genesets_matched_controls/hPPIN*")
hp_genesets_matched = {}

for filename in filenames:
    name = filename.split("hPPIN_")[1][:-4]
    hp_genesets_matched[name] = list()

    with open(filename) as IN:
        for line in IN:
            hp_genesets_matched[name].append(line.rstrip().split("\t")[0])

In [33]:
a_mean = np.zeros(shape=(len(hp_genesets), len(cancersea)))
a_median = np.zeros(shape=(len(hp_genesets), len(cancersea)))
a_dist = pd.DataFrame(index = range(len(cancersea)),
                      columns = range(len(hp_genesets)))

In [34]:
i=0
for key1,value1 in hp_genesets.items():
    j=0
    for key2,value2 in cancersea.items():
        current_info = compute_sp(value1, value2, paths)
        a_mean[i][j] = current_info["mean"]
        a_median[i][j] = current_info["median"]
        a_dist[i][j] = current_info["distribution"]
        j+=1
    i+=1

In [35]:
pd.DataFrame(a_median)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [36]:
pd.DataFrame(a_mean)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0.999835,0.999820,0.999630,0.999786,0.999655,0.999903,0.999654,0.999841,0.999609,0.999694,0.999857,0.999635,0.999972,0.999794,0.999900
1,0.999884,0.999840,0.999823,0.999913,0.999852,0.999938,0.999958,0.999939,0.999927,0.999838,0.999922,0.999838,0.999978,0.999909,0.999928
2,0.999873,0.999757,0.999477,0.999858,0.999709,0.999834,0.999832,0.999819,0.999870,0.999529,0.999868,0.999683,0.999903,0.999724,0.999912
3,0.999608,0.999986,0.999621,0.999691,0.999574,1.000000,0.999777,0.999908,0.999630,0.999710,0.999789,0.999680,0.999983,0.999417,0.999825
4,0.999429,0.999751,0.999740,0.999422,0.999449,0.999561,0.999907,0.999821,0.999892,0.999805,0.999731,0.999642,0.999952,0.999677,0.999741
5,0.999910,0.999800,0.999688,0.999820,0.999731,0.999857,0.999814,0.999802,0.999806,0.999805,0.999892,0.999713,0.999961,0.999871,0.999926
6,0.999852,0.999918,0.999718,0.999886,0.999756,0.999924,0.999708,0.999894,0.999745,0.999770,0.999871,0.999577,0.999952,0.999793,0.999901
7,0.999746,0.999903,0.999825,0.999759,0.999717,0.999659,0.999805,0.999737,0.999754,0.999863,0.999910,0.999782,0.999779,0.999830,0.999835
8,0.999925,0.999981,0.999941,0.999971,0.999906,0.999950,0.999979,0.999959,0.999975,0.999970,0.999904,0.999870,0.999978,0.999926,0.999947


In [37]:
pd.DataFrame(a_mean, columns = cancersea.keys(), index = hp_genesets.keys()).to_csv("STRING_NS_meanSP_HP_Phenotypes.csv")

In [38]:
a_matched_mean = np.zeros(shape=(len(hp_genesets_matched), len(cancersea)))
a_matched_median = np.zeros(shape=(len(hp_genesets_matched), len(cancersea)))
a_matched_dist = pd.DataFrame(index = range(len(cancersea)),
                              columns = range(len(hp_genesets_matched)))

In [39]:
i=0
for key1,value1 in hp_genesets_matched.items():
    j=0
    for key2,value2 in cancersea.items():
        current_info = compute_sp(value1, value2, paths)
        a_matched_mean[i][j] = current_info["mean"]
        a_matched_median[i][j] = current_info["median"]
        a_matched_dist[i][j] = current_info["distribution"]
        j+=1
    i+=1

KeyboardInterrupt: 

In [ ]:
pd.DataFrame(a_matched_median)

In [ ]:
pd.DataFrame(a_matched_mean)

In [ ]:
pd.DataFrame(a_matched_mean, columns = cancersea.keys(), index = hp_genesets_matched.keys()).to_csv("STRING_NS_meanSP_HPmatched_Phenotypes.csv")

In [ ]:
mannwhitneyu_p = np.zeros(shape=(len(hp_genesets), len(cancersea)))
test_dist = np.zeros(shape=(len(hp_genesets), len(cancersea)))
test_matched_dist = np.zeros(shape=(len(hp_genesets), len(cancersea)))

In [ ]:
for i in range(len(hp_genesets)):
    for j in range(len(cancersea)):
        actual_dist = a_dist[i][j]
        control_dist = a_matched_dist[i][j]
        res = mannwhitneyu(actual_dist, control_dist, alternative="less")
        mannwhitneyu_p[i][j] = res.pvalue
        test_dist[i][j] = statistics.mean(actual_dist)
        test_matched_dist[i][j] = statistics.mean(control_dist)

In [21]:
pvals = pd.DataFrame(mannwhitneyu_p, columns = cancersea.keys(), index = hp_genesets.keys())

In [22]:
pvals

,Differentiation,Cell_Cycle,EMT,Proliferation,Metastasis,DNA_damage,Quiescence,Apoptosis,Inflammation,Invasion,Stemness,Hypoxia,DNA_repair,Angiogenesis,COSMIC
downregulated.wound,1.490386e-229,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.239928e-63,3.213933e-130,2.092986e-164,0.000000e+00,1.265250e-284,4.752114e-192,5.160541e-44,2.962549e-01,3.974307e-297,0.000000e+00
upregulated.stress,1.188360e-17,4.750632e-15,2.092735e-37,5.668084e-14,4.918088e-48,3.769377e-01,3.412697e-10,2.206009e-10,3.930779e-21,2.117380e-36,1.059101e-07,8.893450e-05,1.000000e+00,7.005323e-30,1.090844e-12
downregulated.regen,1.604993e-121,4.804168e-99,4.698332e-246,1.753979e-98,1.798448e-235,1.086334e-35,1.468817e-57,6.661684e-64,4.062894e-79,7.937000e-228,2.078579e-80,1.765632e-54,6.035410e-18,1.007072e-172,0.000000e+00
msigdb.wound,5.884858e-187,4.655259e-122,0.000000e+00,1.968735e-151,0.000000e+00,1.024510e-40,2.841708e-81,1.434786e-93,2.309652e-136,0.000000e+00,3.734916e-127,5.049625e-63,3.062994e-07,1.237862e-252,0.000000e+00
msigdb.regen,6.354433e-02,9.999969e-01,2.044952e-20,8.599673e-01,3.692000e-08,9.999995e-01,3.308519e-03,1.813474e-01,6.874445e-02,6.759490e-21,9.504079e-01,9.687007e-03,1.000000e+00,4.096907e-13,9.999674e-01
upregulated.regen,5.505973e-146,8.892092e-254,2.330794e-242,2.382627e-168,0.000000e+00,1.512375e-51,3.446698e-66,1.535704e-84,1.340495e-128,1.107175e-214,8.237998e-116,2.039074e-42,5.243941e-12,6.233189e-174,0.000000e+00
upregulated.wound,3.621788e-49,2.768727e-02,3.044369e-115,3.124210e-22,5.691441e-127,4.169312e-03,9.976000e-24,1.147815e-22,2.954838e-28,9.442701e-130,1.176920e-21,2.461554e-26,9.999857e-01,3.229885e-79,1.118314e-78
msigdb.stress,5.735062e-24,1.141445e-09,3.542512e-63,3.719322e-17,1.801908e-55,1.163352e-05,8.526772e-13,6.676549e-15,4.672663e-19,2.579539e-68,2.466185e-10,1.733848e-14,1.252614e-01,1.080135e-43,8.244394e-67
downregulated.stress,5.911242e-12,9.707517e-01,1.246257e-33,2.039022e-03,2.033070e-27,2.968761e-01,4.490637e-05,6.066004e-04,4.354485e-02,7.479380e-40,8.325505e-07,7.557881e-08,9.979916e-01,2.726762e-18,1.849782e-12


In [23]:
pvals.to_csv("STRING_NS_matched_pvals.csv")

In [24]:
pd.DataFrame(test_dist)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,6.340274,5.542347,5.112914,5.155070,5.309007,6.150527,5.354604,5.179759,5.425850,5.094814,5.733142,6.782794,6.297200,5.191580,5.893264
1,6.672151,5.994683,5.487742,5.668905,5.774245,6.436180,5.964075,5.663859,6.347882,5.439227,6.112262,7.072618,6.404557,5.661782,6.188369
2,6.269818,5.687830,4.810255,5.282640,5.257053,6.156630,5.397984,5.250446,5.921566,4.738512,5.734970,6.490023,6.187481,5.007423,5.827777
3,6.245083,5.868314,4.829321,5.297896,5.195087,6.235993,5.325710,5.217137,5.736241,4.744142,5.724272,6.498960,6.320776,4.919730,5.837475
4,6.617888,6.136773,5.179398,5.737308,5.679059,6.516440,5.884259,5.689994,6.539401,5.094983,6.117365,6.849723,6.496158,5.429298,6.185750
5,6.405481,5.542659,5.188374,5.268011,5.400924,6.141838,5.559274,5.307751,5.795691,5.161753,5.792372,6.845542,6.234454,5.353945,5.947032
6,6.493917,6.009382,5.182284,5.528777,5.497423,6.317534,5.655492,5.458905,6.093158,5.089726,5.944343,6.716292,6.353450,5.348656,6.025648
7,6.673130,5.971661,5.429654,5.632581,5.748086,6.320924,5.915574,5.584010,6.304387,5.372571,6.088608,6.976306,6.243229,5.625164,6.097316
8,6.776463,6.278851,5.627135,5.857923,5.913984,6.527521,6.121839,5.813408,6.560049,5.546325,6.213592,7.130094,6.472530,5.823207,6.294004


In [25]:
pd.DataFrame(test_matched_dist)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,6.922281,6.230053,5.867431,5.913787,6.106067,6.511840,6.281088,5.909658,6.644341,5.819267,6.299884,7.330318,6.355414,6.056215,6.334769
1,6.867314,6.189702,5.802437,5.872410,6.074876,6.476634,6.247499,5.885516,6.675552,5.758429,6.241724,7.264989,6.289845,6.019388,6.284742
2,6.918755,6.338413,5.903356,6.016903,6.142593,6.664976,6.360414,6.043287,6.798444,5.842570,6.340810,7.395556,6.564184,6.139259,6.457824
3,7.053442,6.423809,5.989704,6.085421,6.250140,6.687537,6.428351,6.079138,6.845940,5.939117,6.439332,7.452615,6.534823,6.179058,6.482052
4,6.620000,5.983243,5.605000,5.672194,5.881739,6.246087,6.116000,5.720000,6.593600,5.553385,5.979239,7.042000,5.992340,5.871419,6.054101
5,7.085508,6.364253,6.122312,6.082827,6.322025,6.676049,6.499232,6.117320,6.810612,6.075691,6.448224,7.591244,6.542034,6.337531,6.530237
6,6.815552,5.976996,5.723374,5.730600,5.975196,6.358297,6.134234,5.779133,6.488293,5.696663,6.155045,7.225574,6.234764,5.919532,6.219710
7,6.897830,6.151170,5.847125,5.889112,6.102687,6.501241,6.290695,5.907511,6.672699,5.818629,6.272652,7.342188,6.345922,6.074147,6.327835
8,6.970438,6.285228,5.968662,5.969786,6.188771,6.586377,6.336692,5.978474,6.653562,5.926662,6.362036,7.421252,6.451881,6.152597,6.420270
